##### Table of Content

1. Intro
2. Solving a Sudoku
3. Setting up the Board
4. [Encoding the Board](#cell_encoding_the_board)
5. [Strategy1: Elimination](#cell_elimination)
6. [Strategy2: Only Choice](#cell_only_choice)
7. [Constraint Propagation](#cell_constraint_propagation)
8. [Harder Sudoku](#cell_harder_sudoku)
9. [Strategy3: Search](#cell_search)

** Goals of this project **

The main goal of this project is to build an intelligent agent that will solve every sudoku while introducing you to two powerful techniques that are used throughout the field of AI:

** Constraint Propagation ** 

When trying to solve a problem, you'll find that there are some local constraints to each square. These constraints help you narrow the possibilities for the answer, which can be very helpful. We will learn to extract the maximum information out of these constraints in order to get closer to our solution. Additionally, you'll see how we can repeatedly apply simple constraints to iteratively narrow the search space of possible solutions. Constraint propagation can be used to solve a variety of problems such as calendar scheduling, and cryptographic puzzles.

** Search **

In the process of problem solving, we may get to the point where two or more possibilities are available. What do we do? What if we branch out and consider both of them? Maybe one of them will lead us to a position in which three or more possibilities are available. Then, we can branch out again. At the end, we can create a whole tree of possibilities and find ways to traverse the tree until we find our solution. This is an example of how search can be used.
These ideas may seem simple and they're actually intended to be! Through this lesson you'll see how AI is really composed of very simple ideas that can be put together to solve complex problems. Throughout this lesson, we challenge you to think of how you can apply these ideas to build AI agents to solve other puzzles and problems in your world!

<a id="cell_encoding_the_board"></a>
## Step1: Encoding the Sudoku Board  

In [32]:
'''
Exercise1: Encoding the Board
>>> from utils import display
>>> display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))
  1 2 3  4 5 6  7 8 9
A . . 3 |. 2 . |6 . . 
B 9 . . |3 . 5 |. . 1 
C . . 1 |8 . 6 |4 . . 
  ------+------+------
D . . 8 |1 . 2 |9 . . 
E 7 . . |. . . |. . 8 
F . . 6 |7 . 8 |2 . . 
  ------+------+------
G . . 2 |6 . 9 |5 . . 
H 8 . . |2 . 3 |. . 9 
I . . 5 |. 1 . |3 . .
'''

#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    '''
    Helper Function
    Input: String rows, cols
    Output: String list of keys in each box
    '''
    return [s+t for s in a for t in b]

#1.4 create boxes
# return ['A1', 'B1', 'C1', 'D1', ...]
boxes = cross(rows, cols)

#1.5 create row_units
# return [['A1', 'A2', 'A3', 'A4',...],['B1', 'B2', 'B3', ...], ...]
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
# return [['A1', 'B1', 'C1', 'D1',...],['A2', 'B2', 'C2', ...], ...]
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
# return square block (the board consists of 9 block) of sudoku
# return [['A1', 'A2', 'A3', 'B1', 'B2', 'B3',..], ['D1', 'D2', 'D3', ...]]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
# return {'A1': [[row_units which contain 'A1'],[column_units which contain 'A1'], [square which contains 'A1']]}
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
# return {'A1': set of row, column and squre which contain A1 but exclude 'A1' }
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return
    

#2. function.py ----------------------------
'''
Instruction: create grid_values(grid) A function to convert the string representation 
of a puzzle into a dictionary form.
'''
#2.1 grid_values function (input> strong of sudoku problem, output> dictionary of a suduku with corresponding boxes) 
#from utils import *  #remove this when use as a file
def grid_values(grid):
    # In this function, you will take a sudoku as a string
    # and return a dictionary where the keys are the boxes,
    # for example 'A1', and the values are the digit at each
    # box (as a string) or '.' if the box has no value
    # assigned yet.
    
    ''' Your solution here 
    
    Input : String grid
    Output: Dict boxesVal (Dictionary that contains keys of boxes and values of each index)
    
    '''
    assert len(grid) == 81, 'Input may wrong, please check again'
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    boxesVal = dict([(row_units_long[k[0]], k[1]) for k in enumerate(grid)])
    
    return boxesVal
    
    
    
#3. Test function.py ----------------------------    
# print(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

#4. Test utils.py ---------------------------- 
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


<a id="cell_elimination"></a>
## Step2: Strategy1: Elimination
### <font color='red'> If a box has a value assigned, then none of the peers of this box can have this value. </font>

#### 2.1 First things first, let's look at a box and analyze the values that could go in there.
<img src="images/reduce-values-2.png" width="40%" height="40%"> 

#### 2.2 Elimination strategy
Now that we know how to eliminate values, we can take one pass, go over every box that has a value, and **eliminate the values that can't appear on the box, based on its peers**. Once we do so, the board looks like this 
<img src="images/values-easy.png" width="40%" height="40%">


<font color='red'>** This seems like something we can code!** </font>


In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/6rFOX2jHB2g" frameborder="0" allowfullscreen></iframe>

In [33]:
'''
Exercise2.1: Improved grid_values()

As of now, we are recording the puzzles in dictionary form, where the keys are the boxes ('A1', 'A2', ... , 'I9') 
and the values are either the value for each box (if a value exists) or '.' (if the box has no value assigned yet). 
What we really want is for each value to represent all the available values for that box. 
For example, the box in the second row and fifth column above will have key 'B5' and value '47' 
(because 4 and 7 are the only possible values for it). The starting value for every empty box will thus be '123456789'.
Update the grid_values() function to return '123456789' instead of '.' for empty boxes.

>>> from utils import display
>>> display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3     123456789 123456789 
'''

#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == 81, 'Input may wrong, please check again'
    
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    all_num = '123456789'
    
    boxesVal = dict([(row_units_long[k[0]], k[1] if k[1] != '.' else all_num ) for k in enumerate(grid)])
    
    return boxesVal

#3. Test function.py ----------------------------  
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [16]:
'''
Exercise2.2: implement eliminate()

Now, let's finish the code for the function eliminate(), which will take as input a puzzle in dictionary form. 
The function will iterate over all the boxes in the puzzle that only have one value assigned to them, 
and it will remove this value from every one of its peers.

'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == 81, 'Input may wrong, please check again'
    
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    all_num = '123456789'
    
    boxesVal = dict([(row_units_long[k[0]], k[1] if k[1] != '.' else all_num ) for k in enumerate(grid)])
    
    return boxesVal

#########################
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    values_c = values.copy()
    
    # Loop through all the boxes of sudoku
    for key in values_c.keys():
        # if the value in this key is non-unique, then this value must be eliminated
        
        if len(values_c[key]) > 1:
            for peer in peers[key]:
                # eliminate non-unique by subtract with unique value 
                if len(values_c[peer]) == 1:
                    values_c[key] = values_c[key].replace(values_c[peer],'')
    return values_c

#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
values = eliminate(values)
print("\n")
print("After implement eliminate(values) method **********************************")
display(values)

The original Sudoku board is **********************************************
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9

<a id="cell_only_choice"></a>
## Step3: Only Choice
### <font color='red'> Insight: Every unit must contain exactly one occurrence of every number </font>

#### 3.1 Look more carefully at the top 3x3 square in the center, highlighted in red.
<img src="images/highlighted-unit.png" width="40%" height="40%"> 

#### 3.2 Only Choice Strategy
<font color='red'> ** If there is only one box in a unit which would allow a certain digit, then that box must be assigned that digit. ** </font>

<img src="images/only-choice.png" width="40%" height="40%">

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/sSjYn-Kex1A" frameborder="0" allowfullscreen></iframe>

In [34]:
'''
Exercise3.1: implement only_choice()
Time to code it! In the next quiz, finish the code for the function only_choice, 
which will take as input a puzzle in dictionary form. The function will go through all the units, 
and if there is a unit with a digit that only fits in one possible box, it will assign that digit to that box.
'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == 81, 'Input may wrong, please check again'
    
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    all_num = '123456789'
    
    boxesVal = dict([(row_units_long[k[0]], k[1] if k[1] != '.' else all_num ) for k in enumerate(grid)])
    
    return boxesVal

#########################
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    values_c = values.copy()
    
    # Loop through all the boxes of sudoku
    for key in values_c.keys():
        # if the value in this key is non-unique, then this value must be eliminated
        
        if len(values_c[key]) > 1:
            for peer in peers[key]:
                # eliminate non-unique by subtract with unique value 
                if len(values_c[peer]) == 1:
                    values_c[key] = values_c[key].replace(values_c[peer],'')
    return values_c

#2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    vals_out = values.copy()
    
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                vals_out[dplaces[0]] = digit
    return vals_out
            
            
#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)
values = eliminate(values)
print("\n")
print("After implement eliminate(values) method **********************************")
display(values)

#4. Test function.py ----------------------------  
new_values = only_choice(values)
print("\n")
print("After implement only_choice(values) method **********************************")
display(new_values)

The original Sudoku board is **********************************************
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9

<a id="cell_constraint_propagation"></a>
## Step4: Constraint Propagation
### <font color='red'> using local constraints in a space to dramatically reduce the search space. </font>

#### 4.1 General explanation of constraint propagation
As we enforce each constraint, we see how it introduces new constraints for other parts of the board 
that can help us further reduce the number of possibilities. 

##### 4.1.1 Map Coloring Example 
##### <font color='red'> No two adjacent items can be the same color in the map coloring problem. </font>
<img src="images/map-coloring.jpg" width="40%" height="40%"> 

In the map coloring problem, we must find a way to color the map such that no two adjacent items share the same color. Indeed, we'll see how we use constraint propagation to use this simple constraint to find a solution just as we use such constraints to solve Sudoku.

##### 4.1.2 Crypto-Arithmetic Puzzles 
##### <font color='red'> what digits do T, W, O, F, U, and R represent? </font>
<img src="images/crypto_arithmetic_puzzle.png" width="40%" height="40%"> 

In Crypto-Arithmetic puzzles, each letter represents a digit, and no two letters represent the same digit. None of the numbers start with a leading zero. Our goal is to find a mapping from letters to digits that satisfies the equations. Here again, we'll find that the constraints imposed by the equation allow us to create an intelligent algorithm to solve the problem via Constraint Propagation.

#### 4.2 Applying Constraint Propagation to Sudoku
<font color='red'> ** combine the functions eliminate() and only_choice() to write the function reduce_puzzle(), which receives as input an unsolved puzzle and applies our two constraints repeatedly in an attempt to solve it. ** </font>

** Some things to watch out for: **

- The function needs to stop if the puzzle gets solved. How to do this?
- What if the function doesn't solve the sudoku? Can we make sure the function quits when applying the two strategies stops making progress?

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/aSYDBcbvC5Y" frameborder="0" allowfullscreen></iframe>

In [35]:
'''
Exercise4.1: Apply Constraint Propagation to Sudoku problem
Now that you see how we apply Constraint Propagation to this problem, let's try to code it! 
In the following quiz, combine the functions eliminate and only_choice to write the function reduce_puzzle, 
which receives as input an unsolved puzzle and applies our two constraints repeatedly in an attempt to solve it.

Some things to watch out for:
- The function needs to stop if the puzzle gets solved. How to do this?
- What if the function doesn't solve the sudoku? Can we make sure the function quits when applying 
the two strategies stops making progress?
'''

#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == 81, 'Input may wrong, please check again'
    
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    all_num = '123456789'
    
    boxesVal = dict([(row_units_long[k[0]], k[1] if k[1] != '.' else all_num ) for k in enumerate(grid)])
    
    return boxesVal

#########################
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    values_c = values.copy()
    
    # Loop through all the boxes of sudoku
    for key in values_c.keys():
        # if the value in this key is non-unique, then this value must be eliminated
        
        if len(values_c[key]) > 1:
            for peer in peers[key]:
                # eliminate non-unique by subtract with unique value 
                if len(values_c[peer]) == 1:
                    values_c[key] = values_c[key].replace(values_c[peer],'')
    return values_c

#2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    vals_out = values.copy()
    
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                vals_out[dplaces[0]] = digit
    return vals_out

#2. function.py ----------------------------
# 2.1 combine the functions eliminate and only_choice to write the function reduce_puzzle
# from utils import *
def reduce_puzzle(values_c):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    values = values_c.copy()
    
    no_update = False
    while not no_update:
        # eliminate values
        values_e = eliminate(values)       
        # fill only choices
        new_values = only_choice(values_e)
        
        values_before = len([index for index in values.keys() if len(values[index]) == 1])
        values_after = len([index for index in new_values.keys() if len(new_values[index]) == 1])
        
        display(new_values)
        if '' in new_values.values():
            print('Exit reduce_puzzle')
            print('Error, There is empty possible values in some index')
            return False
        
        if check_complete_sudoku(new_values):
            print('Exit reduce_puzzle')
            print('Complete', check_complete_sudoku(new_values))
            return new_values
        
        # identify if this loop provide no more update
        no_update = values_before == values_after
        values = new_values.copy()
    
    print('Exit reduce_puzzle')
    print('No Update', no_update)
    return new_values

######################## End reduce_puzzle ######################

#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
new_values = reduce_puzzle(values)
print("\n")
print("After applying constrint propagaton (both eliminate and only_choice strategies)*****************")
display(new_values)

The original Sudoku board is **********************************************
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9

##  <font color='red'> So, that seemed to work! Congratulation You should have got this answer. </font>
<img src="images/easy-solution.png" width="40%" height="40%"> 

<a id="cell_harder_sudoku"></a>
## Step5: Harder Sudoku
### <font color='red'> Ok, let's see if our algorithm will work all the time. Here's a harder sudoku puzzle </font>

<img src="images/harder-puzzle.png" width="40%" height="40%"> 

In [36]:
'''
Exercise5.1: Try to solve harder sudoku using the same constraint propagation
'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == 81, 'Input may wrong, please check again'
    
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    all_num = '123456789'
    
    boxesVal = dict([(row_units_long[k[0]], k[1] if k[1] != '.' else all_num ) for k in enumerate(grid)])
    
    return boxesVal

#########################
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    values_c = values.copy()
    
    # Loop through all the boxes of sudoku
    for key in values_c.keys():
        # if the value in this key is non-unique, then this value must be eliminated
        
        if len(values_c[key]) > 1:
            for peer in peers[key]:
                # eliminate non-unique by subtract with unique value 
                if len(values_c[peer]) == 1:
                    values_c[key] = values_c[key].replace(values_c[peer],'')
    return values_c

#2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    vals_out = values.copy()
    
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                vals_out[dplaces[0]] = digit
    return vals_out

#2. function.py ----------------------------
# 2.1 combine the functions eliminate and only_choice to write the function reduce_puzzle
# from utils import *
def reduce_puzzle(values_c):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    values = values_c.copy()
    
    no_update = False
    while not no_update:
        # eliminate values
        values_e = eliminate(values)       
        # fill only choices
        new_values = only_choice(values_e)
        
        values_before = len([index for index in values.keys() if len(values[index]) == 1])
        values_after = len([index for index in new_values.keys() if len(new_values[index]) == 1])
        
        display(new_values)
        if '' in new_values.values():
            print('Exit reduce_puzzle')
            print('Error, There is empty possible values in some index')
            return False
        
        if check_complete_sudoku(new_values):
            print('Exit reduce_puzzle')
            print('Complete', check_complete_sudoku(new_values))
            return new_values
        
        # identify if this loop provide no more update
        no_update = values_before == values_after
        values = new_values.copy()
    
    print('Exit reduce_puzzle')
    print('No Update', no_update)
    return new_values

######################## End reduce_puzzle ######################

#3. Test utils.py ----------------------------  
grid_easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_hard = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid_hard)
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
new_values = reduce_puzzle(values)
print("\n")
print("After applying constrint propagaton (both eliminate and only_choice strategies)*****************")
display(new_values)

The original Sudoku board is **********************************************
    4     123456789 123456789 |123456789 123456789 123456789 |    8     123456789     5     
123456789     3     123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |    7     123456789 123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     2     123456789 |123456789 123456789 123456789 |123456789     6     123456789 
123456789 123456789 123456789 |123456789     8     123456789 |    4     123456789 123456789 
123456789 123456789 123456789 |123456789     1     123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    6     123456789     3     |123456789     7     123456789 
    5     123456789 123456789 |    2     123456789 123456789 |123456789 123456789 12345

##  <font color='red'> Oh no! The algorithm didn't solve it. It seemed to reduce every box to a number of possibilites, but it won't go farther than that. We need to think of other ways to improve our solution. </font>
<img src="images/harder-sudoku-reduced.png" width="40%" height="40%"> 

<a id="cell_search"></a>
## Step6: Strategy 3: Search
### <font color='red'> Search is used throughout AI from Game-Playing to Route Planning to efficiently find solutions. </font>

#### 6.1 An example of Search being used in Google's AlphaGo paper.
Learn more about Google's AlphaGo paper here: [Mastering the game of Go with deep neural networks and tree search](https://storage.googleapis.com/deepmind-media/alphago/AlphaGoNaturePaper.pdf) 
<img src="images/search_alpha_go.png" width="70%" height="70%"> 

#### 6.2 Search Strategy
##### <font color='red'> Pick a box with a minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively. </font>

Before we dive in to code the search function, let's first check our understanding. How would you traverse the following tree using Depth First Search?

<img src="images/bfs-quiz.png" width="40%" height="40%"> 

** DFS Quiz ** 
Traverse the above tree using Depth First Search. The answer should be the string obtained by the labels in the order you've traversed the tree. For example, if your tree has four vertices, A, B, C, D, and you've traversed them in the order B->C->A->D, then the answer should be the string 'BCAD'.

** DFS Answer ** >> 'ABDEHIJCFGKL'

##### <font color='red'> In our example, the box 'G2' has two possibilities: 8 and 9. Why don't we fill it in with a 8 and try to solve our puzzle. </font>
<img src="images/choices.png" width="40%" height="40%"> 

In [ ]:
%%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/BzwTKFbpnXE" frameborder="0" allowfullscreen></iframe>

In [38]:
'''
Exercise6.1: Coding the Solution

Time to code the final solution! Finish the code in the function search, 
which will create a tree of possibilities and traverse it using DFS until it finds a solution for the sudoku puzzle.
'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert len(grid) == 81, 'Input may wrong, please check again'
    
    row_units_long = sum(row_units,[]) # concatenate each list of row_units to 1 long list
    all_num = '123456789'
    
    boxesVal = dict([(row_units_long[k[0]], k[1] if k[1] != '.' else all_num ) for k in enumerate(grid)])
    
    return boxesVal

#########################
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    values_c = values.copy()
    
    # Loop through all the boxes of sudoku
    for key in values_c.keys():
        # if the value in this key is non-unique, then this value must be eliminated
        
        if len(values_c[key]) > 1:
            for peer in peers[key]:
                # eliminate non-unique by subtract with unique value 
                if len(values_c[peer]) == 1:
                    values_c[key] = values_c[key].replace(values_c[peer],'')
    return values_c

#2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    vals_out = values.copy()
    
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                vals_out[dplaces[0]] = digit
    return vals_out

#2. function.py ----------------------------
# 2.1 combine the functions eliminate and only_choice to write the function reduce_puzzle
# from utils import *
def reduce_puzzle(values_c):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    values = values_c.copy()
    
    no_update = False
    while not no_update:
        # eliminate values
        values_e = eliminate(values)       
        # fill only choices
        new_values = only_choice(values_e)
        
        values_before = len([index for index in values.keys() if len(values[index]) == 1])
        values_after = len([index for index in new_values.keys() if len(new_values[index]) == 1])
        
        display(new_values)
        if '' in new_values.values():
            print('Exit reduce_puzzle')
            print('Error, There is empty possible values in some index')
            return False
        
        if check_complete_sudoku(new_values):
            print('Exit reduce_puzzle')
            print('Complete', check_complete_sudoku(new_values))
            return new_values
        
        # identify if this loop provide no more update
        no_update = values_before == values_after
        values = new_values.copy()
    
    print('Exit reduce_puzzle')
    print('No Update', no_update)
    return new_values

######################## End reduce_puzzle ######################

#############################################################
# Additional Method
def check_complete_sudoku(values):
    '''
    This method will check that this sudoku are complete or not

    Input: Dictionary Sudoku
    Output: Boolean out
    '''

    # Loop through row of sudoku
    for row in row_units:
        temp = set([str(x) for x in range(1,10)])
        for index in row:
            if len(values[index]) == 1:          # if not non-unique or not empty
                temp = temp - set(values[index]) # delete value from the temp
            else:
                return False
        # if in row doesn't contain 1 to 9 return false    
        if len(temp) != 0:
            return False

    # Loop through column in sudoku
    for col in column_units:
        temp = set([str(x) for x in range(1,10)])
        for index in col:
            if len(values[index]) == 1:          # if not non-unique or not empty
                temp = temp - set(values[index]) # delete value from the temp
            else:
                return False
        # if in row doesn't contain 1 to 9 return false    
        if len(temp) != 0:
            return False

    # Loop through square in sudoku
    for sq in column_units:
        temp = set([str(x) for x in range(1,10)])
        for index in sq:
            if len(values[index]) == 1:          # if not non-unique or not empty
                temp = temp - set(values[index]) # delete value from the temp
            else:
                return False
        # if in row doesn't contain 1 to 9 return false    
        if len(temp) != 0:
            return False

    return True # pass all of those then return true
############# end check_complete_sudoku ##################
    
#2. function.py ----------------------------
# 2.1 implement search() using Depth First Search Algorithm
#from utils import *

count = 0 # index for counting iteration
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    # Search and Choose one of the unfilled squares with the fewest possibilities
    # Now use recursion to solve each one of the resulting sudokus, 
    # and if one returns a value (not False), return that answer!
    
    import math
    import random
    
    # count iteration of recursive program
    global count
    iter = 10**6
    
    if count < iter:
        
        values_c = values.copy()
        new_values = reduce_puzzle(values_c)

        count += 1

        ## Base Case, search will return False if program cannot solve sudoku
        # or search will return new_values if program can solve sudoku
        if new_values == False:
            return (False, False)
        if check_complete_sudoku(new_values):
            print('root')
            return (True, new_values)
        ##

        # If not complete find new tree leaf
        display(new_values)
        print('gg',check_complete_sudoku(new_values))

        # Loop through all leaf in sort_key
        #for len_val, key in sort_key:
        n,key = min((len(new_values[key]), key) for key in boxes if len(new_values[key]) > 1)
        choices = new_values[key]
        len_choice = len(choices)

        for val in choices:
            print('iter',count,'key',key,'val',val)
            # record top node state
            values_test = new_values.copy()

            # send only copy of the node sate that replace possible value in key
            values_test[key] = val
            (flag, values_test) = search(values_test)

            ## if search return values or flag == True
            # successively return values to the output of first state
            if flag == True:
                print('key',key,': val', val)
                return (True, values_test)

    return (False, values) # more than iteration limits
    ################################
    
#3. Test utils.py ----------------------------  
grid_easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_normal = '.4.59.......6.7...2....18..4.5...36..1.....4..96...2.7..71....4...9.3.......82.3.'
grid_hard = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid_hard)
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
(flag, new_values) = search(values)
print("\n")
print("After applying Depth First Search Algorithm *****************")
print(flag)

if flag:
    print('Found a solutions!')
    display(new_values)
else:
    print('Solutions not found')





The original Sudoku board is **********************************************
    4     123456789 123456789 |123456789 123456789 123456789 |    8     123456789     5     
123456789     3     123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |    7     123456789 123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     2     123456789 |123456789 123456789 123456789 |123456789     6     123456789 
123456789 123456789 123456789 |123456789     8     123456789 |    4     123456789 123456789 
123456789 123456789 123456789 |123456789     1     123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    6     123456789     3     |123456789     7     123456789 
    5     123456789 123456789 |    2     123456789 123456789 |123456789 123456789 12345

  689     59    2568 |   7     2569  245689|  1236  12349  23469 
---------------------+---------------------+---------------------
  3789    2     158  |   4      3     579  |  137     6     3789 
  3679    59    156  |   59     8    25679 |   4    12359   2379 
 36789    4     568  |   59     1    25679 |  237   23589  23789 
---------------------+---------------------+---------------------
   2      8      9   |   6      4      3   |   5      7      1   
   5      6      3   |   2      7      1   |   9      48     48  
   1      7      4   |   8      59     59  |  236     23    236  
   4      1      7   |   3     269    269  |   8      29     5   
  689     3     2568 |   1     2569  245689|  267    249   24679 
  689     59    2568 |   7     2569  245689|  1236  12349  23469 
---------------------+---------------------+---------------------
  789     2     158  |   4      3     579  |   17     6     789  
  3679    59    156  |   59     8    25679 |   4    12359   2379 
 36789    

   5      6      3   |   2      7      1   |   9      48     48  
   1      7      4   |  589     59    589  |  236    238    2368 
   4      9      7   |   3      26     26  |   8      1      5   
   68     3     2568 |   1     2569  245689|  267    249   24679 
   68     15   12568 |   7     2569  245689|  236    2349  23469 
---------------------+---------------------+---------------------
  789     2     158  |   4      3     4579 |   1      6     789  
  3679    15    156  |   59     8    25679 |   4     2359   2379 
 36789    4     568  |   59     1    25679 |  237   23589  23789 
---------------------+---------------------+---------------------
   2      8      9   |   6      4      3   |   5      7      1   
   5      6      3   |   2      7      1   |   9      48     48  
   1      7      4   |   8      59    589  |  236    238    2368 
   4      9      7   |   3      26     26  |   8      1      5   
   68     3     2568 |   1     2569  245689|  267    249   24679 
   68     

##  <font color='red'> So, that seemed to work! Congratulation You should have got this answer. </font>
<img src="images/hard-solution.png" width="40%" height="40%"> 